## Modeling video data with 2DCNN for frame and biGRU for time

Need output data from <b>01 processing videos</b>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, GRU, Attention, Reshape, Concatenate
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.backend as kr
import pickle
from data_split import train_valid_test_split

### Load videos

In [ ]:
####
#parameters
test_rate = 0.15
valid_rate = 0.15
window = 30 #frame window
seed = 123456

####
#load focus videos
of = open('processed_videos_160.obj', 'rb')
data, labels = pickle.load(of)
of.close()

####
#transform classes
from sklearn.preprocessing import OrdinalEncoder
label_nums = OrdinalEncoder().fit_transform(labels)

####
#split into train/valid/test
X_train,X_valid,X_test,y_train,y_valid,y_test = train_valid_test_split(data,label_nums,test_rate,valid_rate,window,seed)

### Model building

In [ ]:
####
#parameters
face_cnn_layers = [[8,4,4], [8,4,4],[8,4,4],[4,4,4]]
gru_size = 64
out_dense = [64]
dropout = False
droprate = 0.4

####
#functional codes
face_in = tf.keras.Input(shape=(window,160,160,1))
x1 = face_in
#Face 2dCNN blocks
x1 = kr.reshape(x1, (-1,x1.shape[2],x1.shape[3],x1.shape[4]))
# facial cnn
for fcnn in face_cnn_layers:
    x1 = Conv2D(fcnn[0],(fcnn[1],fcnn[2]), activation="relu")(x1)
    x1 = MaxPool2D([2,2], padding='same')(x1)
    x1 = BatchNormalization()(x1)
    if dropout:
        x = Dropout(droprate)(x)
x1 = kr.reshape(x1, (-1,window,x1.shape[1]*x1.shape[2]*x1.shape[3]))
x1 = Dense(x1.shape[2]//4, activation='relu')(x1)   
# forward direction
x2 = x1
x2 = GRU(gru_size, return_sequences=True)(x2)
x2 = BatchNormalization()(x2)
# backward direction
x3 = tf.reverse(x1, axis=[1])
x3 = GRU(gru_size, return_sequences=True)(x3)
x3 = BatchNormalization()(x3)
# Self-Attention Block
x = tf.concat([x2,x3], axis=-1)
if dropout:
    x = Dropout(droprate)(x)
x = Attention()([x, x])
x = tf.keras.layers.GlobalMaxPooling1D()(x)
# Output Block
for d in out_dense:
    x = Dense(d, activation="relu")(x)
    if dropout:
        x = Dropout(droprate)(x)
output = Dense(units=1, activation='softmax')(x)
# Build and Compile model
model = Model(inputs=[face_in], outputs=[output])
model.summary()

### Training

In [ ]:
opt = SGD(lr=1e-3, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train_face, y_train, validation_data=[X_valid_face, y_valid], batch_size=256, epochs=1000)

### Testing

In [ ]:
model.evaluate(X_test_face, y_test)